# Artificial Neural Networks and Deep Learning

---

## Homework 1: Hand-in

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 1` in your Google Drive.
2. Upload the `training_set.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 1 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.zip` file to Codabench.


## 🌐 Connect Colab to Google Drive

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/My Drive/[2024-2025] AN2DL/Homework 1

## 📊 Prepare Your Submission

To prepare your submission, create a `.zip` file that includes all the necessary code to run your model. It **must** include a `model.py` file with the following class:

```python
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""
```

The next cell shows an example implementation of the `model.py` file, which includes loading model weights from the `weights.keras` file and conducting predictions on provided input data. The `.zip` file is created and downloaded in the last notebook cell.

❗ Feel free to modify the method implementations to better fit your specific requirements, but please ensure that the class name and method interfaces remain unchanged.

In [1]:
%%writefile model.py
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following initializes the model with the weights from a pre-trained model.
        """
        # Best values found so far
        self.n_dense_layers = 2
        self.dropout_rate = 0.35
        self.include_dropout = True
        self.l2_lambda = 0

        self.model = self._create_model(n_dense_layers=self.n_dense_layers, include_dropout=self.include_dropout, dropout_rate=self.dropout_rate, l2_lambda=self.l2_lambda)
        # Load the model weights if available
        self.model.load_weights('LION_AUGMIX_AUG_MODEL.weights.h5')

    def _create_model(
            self,
            convnext_trainable=False, #standard definitions
            n_dense_layers=1, 
            initial_dense_neurons=1024, 
            min_neurons=64, # architecture definitions
            include_dropout=False, 
            dropout_rate=0.3, 
            l2_lambda=0, # against overfitting
            learning_rate=1e-4):
    
        input_shape = (96, 96, 3)
        output_shape = 8
    
        # The input layer
        inputs = tfkl.Input(shape=input_shape, name='Input')   


        # The convnext layer with include top=False to take the convolutional part only
        convnext = tfk.applications.ConvNeXtXLarge(
                    input_shape=input_shape,
                    weights=None,
                    include_top=False
                )

        # Here we freeze the convnext to perform Tranfer Learning
        convnext.trainable = convnext_trainable

        x = convnext(inputs)
        x = tfkl.GlobalAveragePooling2D()(x)

        # Hidden layers building
        neurons = initial_dense_neurons
        for k in range(n_dense_layers):
            x = tfkl.Dense(units=neurons, activation='silu', name=f'Dense_layer_{k}', kernel_regularizer=tfk.regularizers.L2(l2_lambda))(x)
            if include_dropout:
                x = tfkl.Dropout(dropout_rate, name=f'Dropout_layer_{k}')(x)
            neurons = max(neurons // 2, min_neurons)
        outputs = tfkl.Dense(output_shape, activation='softmax', name='output_layer')(x)

        # Final model building
        model = tfk.Model(inputs=inputs, outputs=outputs, name='TF-CNN')

        # Compile the model
        loss = tfk.losses.CategoricalFocalCrossentropy(
                                                alpha=0.25,
                                                gamma=2.0,
                                                from_logits=False,
                                                label_smoothing=0.0,
                                                axis=-1,
                                                reduction="sum_over_batch_size",
                                                name="categorical_focal_crossentropy",
                                                dtype=None,
                                            )
        # Metrics definition
        #weighted_macro_f1 = WeightedMacroF1Score(num_classes=n_labels)
        METRICS = [tfk.metrics.CategoricalAccuracy()]
        optimizer = tfk.optimizers.Lion(learning_rate)
        
        model.compile(loss=loss, optimizer=optimizer, metrics=METRICS)

        # Return the model
        return model
    
    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 224, 224, 3) and the output should be a numpy
        array of shape (n_samples,). The outputs are not one-hot encoded.

        The following is an example of a prediction from the model.
        """
        preds = self.model.predict(X)
        if len(preds.shape) == 2:  # If the output is one-hot encoded, we convert it
            preds = np.argmax(preds, axis=1)
        return preds

Writing model.py


In [2]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py LION_AUGMIX_AUG_MODEL.weights.h5

  adding: model.py (164 bytes security) (deflated 65%)
  adding: LION_AUGMIX_AUG_MODEL.weights.h5 (164 bytes security) (deflated 7%)


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following initializes the model with the weights from a pre-trained model.
        """
        # Best values found so far
        self.n_dense_layers = 2
        self.dropout_rate = 0.35
        self.include_dropout = True
        self.l2_lambda = 0

        self.model = self._create_model(n_dense_layers=self.n_dense_layers, include_dropout=self.include_dropout, dropout_rate=self.dropout_rate, l2_lambda=self.l2_lambda)
        # Load the model weights if available
        self.model.load_weights('LION_AUGMIX_AUG_MODEL.weights.h5')

    def _create_model(
            self,
            convnext_trainable=False, #standard definitions
            n_dense_layers=1, 
            initial_dense_neurons=1024, 
            min_neurons=64, # architecture definitions
            include_dropout=False, 
            dropout_rate=0.3, 
            l2_lambda=0, # against overfitting
            learning_rate=1e-4):
    
        input_shape = (96, 96, 3)
        output_shape = 8
    
        # The input layer
        inputs = tfkl.Input(shape=input_shape, name='Input')   


        # The convnext layer with include top=False to take the convolutional part only
        convnext = tfk.applications.ConvNeXtXLarge(
                    input_shape=input_shape,
                    weights=None,
                    include_top=False
                )

        # Here we freeze the convnext to perform Tranfer Learning
        convnext.trainable = convnext_trainable

        x = convnext(inputs)
        x = tfkl.GlobalAveragePooling2D()(x)

        # Hidden layers building
        neurons = initial_dense_neurons
        for k in range(n_dense_layers):
            x = tfkl.Dense(units=neurons, activation='silu', name=f'Dense_layer_{k}', kernel_regularizer=tfk.regularizers.L2(l2_lambda))(x)
            if include_dropout:
                x = tfkl.Dropout(dropout_rate, name=f'Dropout_layer_{k}')(x)
            neurons = max(neurons // 2, min_neurons)
        outputs = tfkl.Dense(output_shape, activation='softmax', name='output_layer')(x)

        # Final model building
        model = tfk.Model(inputs=inputs, outputs=outputs, name='TF-CNN')

        # Compile the model
        loss = tfk.losses.CategoricalFocalCrossentropy(
                                                alpha=0.25,
                                                gamma=2.0,
                                                from_logits=False,
                                                label_smoothing=0.0,
                                                axis=-1,
                                                reduction="sum_over_batch_size",
                                                name="categorical_focal_crossentropy",
                                                dtype=None,
                                            )
        # Metrics definition
        #weighted_macro_f1 = WeightedMacroF1Score(num_classes=n_labels)
        METRICS = [tfk.metrics.CategoricalAccuracy()]
        optimizer = tfk.optimizers.Lion(learning_rate)
        
        model.compile(loss=loss, optimizer=optimizer, metrics=METRICS)

        # Return the model
        return model
    
    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 224, 224, 3) and the output should be a numpy
        array of shape (n_samples,). The outputs are not one-hot encoded.

        The following is an example of a prediction from the model.
        """
        preds = self.model.predict(X)
        if len(preds.shape) == 2:  # If the output is one-hot encoded, we convert it
            preds = np.argmax(preds, axis=1)
        return preds

In [2]:
model = Model()

c:\Users\camil\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'lion', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
del model